# Freedmen's Bureau Analysis 

## Task #0: Configure an LLM

### Import libraries that we'll use in this notebook

In [111]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

import json
import pandas as pd
import ast


## Task #1: Parse Entitities from Simple Examples (Warmup)

### Samples to be able to parse correctly

In [117]:

inputs = [
    "John James agrees to pay $50/month to RJ Hampshire for work on the Farm",
    "Elizabeth James will pay $30 per month to Levi Rodgers for Gardening",
    "Johnson Ollaman will pay $1.25 per day to both John Smith and Jane Smith for teaching the children of the community",
    "Claire Daniels charges $50 weekly to local community members for cooking classes, emphasizing the joy of healthy eating.",
    "Marcus Wellby commits to donating $500 annually to the Green Earth Foundation for environmental conservation efforts.",
    "Dr. Helena Russell charges $100 per hour for providing guidance and support to medical students, aiming to enhance their clinical skills and knowledge.",
    "Keith Galli charges $0 to watch his YouTube content; the least you could do is smash that like button and subscribe, hehehe xD",
    "The city council agrees to pay $15,000 to Urban Design Ltd. for the development of the new park designs, to be completed over the next six months.",
    "Samantha Lee offers piano lessons at $30 per session, each session lasting for 45 minutes, available every Tuesday and Thursday.",
    "Global Tech Inc. commits to paying $200,000 for the licensing rights to innovative software developed by Startup Innovations, with payment due upon contract signing.",
    "The local sports club agrees to pay $75 each to coaches Sarah Miller, Danny Glover, Alex Reed, and Jamie Fox for conducting a weekend sports clinic.",
    """This Agreement made this 14th day of August A.D. 1865, by and between F.R.J. Terry of the county of Copiah and State of Mississippi of the first part, and the person hereinafter named and undersigned, Freedmen of the second part [[?]] That for the purpose of working in the [[?]] known as Beagley's [[?]] Yard in the county aforesaid for two months commencing on the 14th day of August 1865 and terminating on the 14th day of October 1865. The said F.R.J. Terry party of the first part, in consideration of the [[?]] and conditions hereinafter mentioned on the part of the party of the second part agrees to pay said laborer "10" ten dollars per month and furnish free of charge clothing and good of good quality and sufficient quantity, good and sufficient quarters, and kind and humane treatment. And it is further agreed that in case the said F.R.J. Terry shall fail, neglect, or refuse to fulfill any of the obligations assumed by him, he shall besides the legal recourse left to the party aggrieved render this contract liable to amendment by the Provost Marshal of Freedmen. And it is agreed on the part of the party of second part that he will well and faithfully perform such labor as the said F.R.J. Terry may require of him for the time aforesaid, nor exceeding ten hours per day in summer and nine hours in winter. And in case the said laborer shall absent himself from or refuse to perform the labor herein promised, he shall loose the time and be punished as such manner as the Provost Marshal shall deem propper.""",
]

outputs = [
    [{"payer": "John James", "recipient": "RJ Hampshire", "amount": 50, "pay frequency": "monthly", "description": "farming"}],
    [{"payer": "Elizabeth James", "recipient": "Levi Rodgers", "amount": 30, "pay frequency": "monthly", "description": "gardening"}],
    [{"payer": "Johnson Ollaman", "recipient": "John Smith", "amount": 1.25, "pay frequency": "daily", "description": "teaching the children of the community"}, {"payer": "Johnson Ollaman", "recipient": "Jane Smith", "amount": 1.25, "pay frequency": "daily", "description": "teaching the children of the community"}],
    {"payer": "Claire Daniels", "recipient": "Local community members", "amount": 50, "pay frequency": "weekly", "description": "cooking classes"},
    {"payer": "Marcus Wellby", "recipient": "Green Earth Foundation", "amount": 500, "pay frequency": "yearly", "description": "donation for environmental conservation"},
    {"payer": "Dr. Helena Russell", "recipient": "Medical students", "amount": 100, "pay frequency": "hourly", "description": "mentorship and clinical skill enhancement"},
    {"payer": None, "recipient": "Keith Galli", "amount": 0, "pay frequency": None, "description": "YouTube content"},
    {"payer": "The city council", "recipient": "Urban Design Ltd.", "amount": 15000, "pay frequency": "one-time", "description": "development of new park designs"},
    {"payer": "Samantha Lee", "recipient": "Students", "amount": 30, "pay frequency": "per session", "description": "piano lessons"},
    {"payer": "Global Tech Inc.", "recipient": "Startup Innovations", "amount": 200000, "pay frequency": "one-time", "description": "licensing rights to software"},
    [{"payer": "The local sports club", "recipient": "Sarah Miller", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Danny Glover", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Alex Reed", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"},
     {"payer": "The local sports club", "recipient": "Jamie Fox", "amount": 75, "pay frequency": "one-time", "description": "weekend sports clinic"}],
     {"payer": "F.R.J. Terry", "payee": "Freedmen", "amount":10, "pay frequency": "monthly", "description": "working in the yard"},
]

### Your code here

In [118]:
system_message = """
  From the following historical document text, please grab out the following items and return with the following format
  [{{
    "payer": "<Payer Name>",
    "recipient": "<Payee Name>",
    "amount": <Amount Paid>, # This is the amount of money in USD
    "pay frequency": "<Frequency Paid>" # Only options are "hourly, "daily", "monthly", "weekly", "yearly", "one-time", or "other"
    "description": "<Description of Work>" # This is the description of the work being done
  }},
  {{<ITEM 2>}}, {{Item 3>}}, ...]

  So for the following text:
  "Jane Doe will pay $50/month to John Smith for work on the Farm"

  the output text generated would be:
  [{{
    "payer": "Jane Doe",
    "recipient": "John Smith",
    "amount": 50,
    "pay frequency": "monthly"
    "description": "work on the farm"
  }}]

  IMPORTANT: Return only the list of Python dictionary objects and nothing more. It is possible there are no matches, in which case return an empty list.
  """

In [119]:
# chat = ChatOpenAI()
chat = ChatOllama(model="llama2")


def get_output(input):
    messages = [SystemMessage(content=system_message), HumanMessage(content=inputs[0]), AIMessage(content=str(outputs[0])), HumanMessage(content=input)]
    output = chat.invoke(messages)
    return output.content

In [120]:
def parse_output(output_str):
    """Parse output from a JSON string to a dictionary."""
    try:
        return ast.literal_eval(output_str)
    except Exception as e:
        print("Error decoding JSON from output")
        return None

def compare_outputs(expected, actual):
    """Compare expected and actual output dictionaries and print comparison results."""
    fields = ["payer", "recipient", "amount", "pay frequency"]
    results = {}
    for field in fields:
        expected_val = expected.get(field)
        actual_val = actual.get(field)
        
        results[field] = (expected_val == actual_val, expected_val, actual_val)
    return results

def test_model_with_examples(chat, inputs, outputs):
    """Test the model with all examples and print the comparison results."""
    for index, example in enumerate(inputs):
        # Simulate model invocation (Replace this with actual model call)
        output_str = get_output(example)
        print(output_str, type(output_str))
        actual_output = parse_output(output_str)

        print(actual_output, type(actual_output))
        
        if actual_output:
            print(f"Testing Example {index + 1}: {example}")
            for i in range(len(actual_output)):
                results = compare_outputs(outputs[index][i], actual_output[i])
                print(results)
                for field, (match, expected, actual) in results.items():
                    status = "Match" if match else f"Mismatch (Expected: {expected}, Actual: {actual})"
                    print(f"  {field}: {status}")
                print("\n" + "-"*50 + "\n")
        else:
            print(f"Invalid output format for example {index + 1}.")

# Example usage (assuming chat is already defined and has an invoke method)
test_model_with_examples(chat, inputs, outputs)

[{'payer': 'John James', 'recipient': 'RJ Hampshire', 'amount': 50, 'pay frequency': 'monthly', 'description': 'work on the Farm'}] <class 'str'>
[{'payer': 'John James', 'recipient': 'RJ Hampshire', 'amount': 50, 'pay frequency': 'monthly', 'description': 'work on the Farm'}] <class 'list'>
Testing Example 1: John James agrees to pay $50/month to RJ Hampshire for work on the Farm
{'payer': (True, 'John James', 'John James'), 'recipient': (True, 'RJ Hampshire', 'RJ Hampshire'), 'amount': (True, 50, 50), 'pay frequency': (True, 'monthly', 'monthly')}
  payer: Match
  recipient: Match
  amount: Match
  pay frequency: Match

--------------------------------------------------

[{'payer': 'Elizabeth James', 'recipient': 'Levi Rodgers', 'amount': 30, 'pay frequency': 'monthly', 'description': 'Gardening'}] <class 'str'>
[{'payer': 'Elizabeth James', 'recipient': 'Levi Rodgers', 'amount': 30, 'pay frequency': 'monthly', 'description': 'Gardening'}] <class 'list'>
Testing Example 2: Elizabeth 

KeyError: 0

## Task #2a: Grab Labor Contract Rows from Kaggle

In [67]:
import pandas as pd

df = pd.read_csv('./input/contract-records.csv')
labor = df[df['sub_category']=='Labor Contracts']
labor.head()

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# pd.reset_option('display.max_colwidth')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_rows')



labor.head()


,project_id,category,sub_category,transcription_text,document_url
10517,15153,Contracts,Labor Contracts,"[[preprinted]]_x000D_\nAGREEMENT FOR LABOR._x000D_\n_x000D_\nTHIS INDENTURE, Made and entered into on the ^[[14]] day of ^[[December]] A.D. 186^[[6]], between ^[[Tacitus Clay]] of the County of ^[[Washington]] and State of [[strikethrough]] North Carolina [[/strikethrough]] ^[[insertion]] Texas [[/insertion]], of the first part, and the undersigned, Free Laborer, of the second part:_x000D_\n_x000D_\nWITNESSETH, That the said Laborers have agreed to work for the said ^[[Clay Twelve Months]] from [[strikethrough]] this date to the day of [[/strikethrough]] th^[[e]] ^[[arrival at Said Clays]] 186^[[7]]; to do all kinds of work common to farms of the country; to do their work faithfully, and to be respectful in their deportment. And they further agree, that they only receive every month, the amount placed opposite their names, and the rest at the end of the year; and that the amount so retained shall be forfeited if a violation of this contract is clearly proven. And they further agree, [[strikethrough]] that time lost by idleness or absence without leave, shall not be paid for. [[/strikethrough]] ^[[to loose no time that can be avoided]]_x000D_\n_x000D_\nAnd the said ^[[Clay]] has agreed that he will furnish the said laborers with comfortable quarters, sufficient rations, and the amount of money per month, which stands opposite their respective names; that he will treat them kindly, and encourage the establishment of schools for their children. [[strikethrough]] ^[[Pay what he considers necessary during said Service and the balance at the expiration of Said Service]] [[/strikethrough]]_x000D_\n_x000D_\n[[table]]_x000D_\nNo. | NAME. | AGE. | ^[[Twelve months]] WAGES. DOLLS. CTS. | MONTHLY PAYM'T. DOLLS. CTS. [[/preprinted]]_x000D_\n|Phil Shaver His x mark|33|$144|00|2|00_x000D_\n|John Leo His x mark|16|72|00|1|50_x000D_\n|Mumford Randleman His x mark|25|120|00|2|00_x000D_\n|Adam Fisher His x mark|25|144|00|2|00_x000D_\n|Emma Fisher Her x mark|19|72|00|2|00_x000D_\n|Joseph Thompson His x mark|19|144|00|2|00_x000D_\n|Henry Kelly Ale His mark x|20|144|00|2|00_x000D_\n|Samuel Gillespie His x mark|22|120|00|2|00_x000D_\n|Nathan Foard His x mark|40|144|00|2|00_x000D_\n|Jordan Cox, His x mark|19|120|00|2|00_x000D_\n|Ann Tyson Her x mark|18|84|00|2|00_x000D_\n|Liza Dixon Her x mark|30|84|00|2|00_x000D_\n|Ceily Auston Her x mark||25|00|84|00_x000D_\n|Leon Foard His x mark|13|72|00|2|00_x000D_\n|Silas Foard His x mark|10|60|00|2|00_x000D_\n|Alfred P. Dixon His x mark|10|48|00|2|00_x000D_\n|Mary Hall Her x mark|25|72|00|2|00_x000D_\n|Hicksey Her x mark|25|72|00|2|00_x000D_\n|Dennis His x mark|14|48|00|2|00_x000D_\n|George Little & His x mark|31|120|00|2|00_x000D_\n|Wife|23|96|00|1|00_x000D_\n|Will Alfred & His x mark|28|120|00|2|00_x000D_\n|Wife|19|96|00|2|00_x000D_\n|Henry Myers His x mark|22|144|00|2|00_x000D_\n|Thomas Granderson His x mark|24|120|00|2|00_x000D_\n|Fred Deaton His x mark|23|120|00|2|00_x000D_\n|[[Illegible - final line of page is cut off]]|14|72|00|2|00_x000D_\n[[/Table]]_x000D_\n[[/Preprinted]]",https://transcription.si.edu/transcribe/15153/NMAAHC-004567412_00046
10518,15153,Contracts,Labor Contracts,"[[table]]_x000D_\nNo. | NAME. | AGE. | ^[[Twelve months]] WAGES. DOLLS. CTS. | MONTHLY PAYM'T. DOLLS. CTS._x000D_\n_x000D_\n |John Finley His x mark| 14 | 72 | 00 | 2 | 00|_x000D_\n_x000D_\n |Wiley Rendleman & His x mark |23 | 120 | 00 | 2 | 00 |_x000D_\n |Wife| 20 | 72 | 00 | 2 | 00 |_x000D_\n_x000D_\n |Hock Lee His x mark |15 | 120| 00| 2| 00 |_x000D_\n_x000D_\n |John Chambers His x mark| 23|144|00|2|00|_x000D_\n |Wife| 21 | 96| 00| 2| 00 |_x000D_\n_x000D_\n |Emanuel Dixon His x mark| 72 | 00| 1| 00|_x000D_\n_x000D_\n |Elizabeth Dixon His x mark | 28 | 48 | 00 | 2 | 0 |_x000D_\n_x000D_\n |Elvira Dixon His x mark | 23 | 96 | 00 | 2 | 00 |_x000D_\n_x000D_\n |Hannah Dixon His x mark | 22 | 96 | 00 | 2 | 00 |_x000D_\n_x000D_

## Task #2: Connect pages that belong to the same document